# Introducing `scipy`

*October 19 2022*

This notebook introduces basic statistical concepts in the package `scipy`, which we will use for statistical data analysis and hypothesis in the following practice lectures.

## Sparse matrix algebra

Naturally extending the introduction of vector and matrix operations in `numpy` in the last unit, we first highlight `scipy`'s support for efficient operations on sparse matrices and vector, i.e. large matrices where most entries are actually zeros. In data science, we often deal with such matrices, be it in the context of network analysis and graph mining (where each non-existing link leads to a zero entry in the adjacency matrix) or when we we deal with matrices capturing similarity scores (where zero entries indicate objects that are either not similar or for which the similarty is not known).

Clearly, it is not efficient to perform mathematical operations on matrices where the majority of entries is zero anyway. The module `scipy.sparse` thus offers classes to represent such matrices. The class `csr_matrix` (CSR = Compressed Sparse Row) for instance can be initialised from a list of `python` lists or, conveniently, from a `numpy` array. As we see below, different from a `numpy` array, the class `csr_matrix` only stores the non-zero entries of the matrix:

In [1]:
import numpy as np
import scipy.sparse

matrix = np.array([[1., 0., 1.], [0., 1., 0.], [0., 0., 1.]])
print(matrix)

sparse_matrix = scipy.sparse.csr_matrix(matrix)
sparse_matrix
print(sparse_matrix)

[[1. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
  (0, 0)	1.0
  (0, 2)	1.0
  (1, 1)	1.0
  (2, 2)	1.0


If we wish to recover a representation with zeros, we can call the `to_dense` method on an instance of `csr_matrix`. This function returns an instance of the class `numpy.matrix`:

In [2]:
print(sparse_matrix.todense())
print(type(sparse_matrix.todense()))

[[1. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
<class 'numpy.matrix'>


We can directly perform vector and matrix operations on sparse matrices, i.e. we can do the following:

In [3]:
v = scipy.sparse.csr_matrix([2,3,1])
w = v.dot(sparse_matrix)
print(w.todense())

[[2. 3. 3.]]


To compute eigenvalues and eigenvectors, we can use the `eigs` function:

In [4]:
import scipy.sparse.linalg

w, v = scipy.sparse.linalg.eigs(sparse_matrix, k=1)
print(w)
print(v)

[1.00000001+0.j]
[[-1.00000000e+00+0.j]
 [ 9.87713611e-12+0.j]
 [-1.05367098e-08+0.j]]


In [5]:
print(sparse_matrix.dot(v[:,0]))
print(w[0]* v[:,0])

[-1.00000001e+00+0.j  9.87713611e-12+0.j -1.05367098e-08+0.j]
[-1.00000001e+00+0.j  9.87713621e-12+0.j -1.05367099e-08+0.j]


## Statistical computing with `scipy.stats`

In our course, we will extensively use `scipy`'s implementation of statistical functions, random variables, and probability distributions to generate random multi-variate data, perform linear regression, or test hypotheses. These functions are implemented in the statistics module `scipy.stats`, which contains an [extensive list of classes](https://docs.scipy.org/doc/scipy/reference/stats.html) that implements random variables distributed according various well-known probability distributions. We can, for instance, create a continuous random variable distributed according to a standard normal distribution (with mean 0 and standard deviation 1) as follows:





In [6]:
import scipy.stats
norm = scipy.stats.norm()

Depending on the domain of the distribution, `scipy` returns an instance of class [`rv_continuous`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.html#scipy.stats.rv_continuous) or [`rv_discrete`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_discrete.html#scipy.stats.rv_discrete). Both classes provide functions that -- among other things -- allow us to (i) generate random realisations of the random variable, (ii) calculate moments of the distribution (i.e. mean, variance, skewness, etc.), and (iii) get the underlying probability distribution or cumulative distribution function.

To generate a `numpy` array of 100 random realisations of the normally distributed random variable, we can do the following:

In [7]:
x = scipy.stats.norm.rvs(size=100)
print(x)

[-0.55741023  0.08254122  0.9580905  -1.51703723  0.44441338  0.99973818
  1.10694576  0.35148339  1.30394998 -0.06168308 -1.72221767  1.55763362
 -0.65582876  0.49494623 -0.07800506  1.09055195 -0.08671544  1.61708779
 -0.02528814 -2.23685192 -1.07194663  0.74571247  0.49928861 -1.8629672
 -2.9492341  -1.64910205 -1.26204654  0.6453319  -1.03770563 -0.04601738
 -0.44752721 -0.18626106 -1.14318422 -1.55767597 -0.23715831  0.82301336
  0.21918765 -0.59829132  1.30200793  0.23021079  0.88711426 -0.1142638
 -0.95271775 -0.65727423  1.4991022   1.39184253 -1.58751053  1.05312521
  0.51181609 -0.29985935 -0.08375893 -1.26259606  1.07572847  0.53686409
  1.22694064  1.42158857 -0.11574819  0.17508692  1.60134722 -1.21081734
  0.54917457  0.18235993  0.76016082 -0.09202542 -1.31320222 -0.82754889
 -0.80556631  1.43087043  0.30678069 -0.83041188 -0.26154206  2.37238945
 -0.34899291 -2.79870984 -0.88848455  0.03797561 -0.60176109 -1.07746863
 -1.91821109  1.69749926  1.46822094  0.58774487  2.0

The above call generates 100 realisations of a normally distributed random variable with mean $\mu=0$ and standard deviation $\sigma^2=1$. A simple way to create realisations drawn from a normal distribution with arbitrary $\mu$ and $\sigma$ is to perform an element-wise addition and multiplication of the resulting `numpy` array. To obtain realisations from a normal distribution with mean $\mu=5$ and standard deviation $\sigma=2$, we can thus write:

In [8]:
x = 5 + 2 * scipy.stats.norm.rvs(size=5000)
print('Sample mean = {0}'.format(np.mean(x)))
print('Sample variance = {0}'.format(np.std(x)))

Sample mean = 5.022393094351404
Sample variance = 1.9898837835117553


Let us try this with a discrete random variable, that is distributed according to the Poisson distibution. We can directly pass the single parameter $\mu$ of the distribution to the `rvs` function.

In [9]:
x = scipy.stats.poisson.rvs(mu=5, size=100)
print(x)

[ 3  5  7  3  2  3  4  8  2  2  5  5  4  5  6  4  4  5  5  2  4  6  5  5
  4  4  5  6  5  6 12  5  4  4  7  9  8  5  6  6  5  6  6  3  8  2  5  6
  4  4  4  2  3  6  4  6  4  8  8  2  8  7  5  5  6  4  3  6  9  2  5  7
  5  9  3  1  3  4  3 10  7  5  2  3  2  8  7  2  3  6  3  7  3  5  6  5
  3  7  6  4]


The following generates 100 realisations of a random variable distributed according to a Binomial distribution with parameters $n$ and $k$. For $n=1$ we obtain a sequence of Bernoulli trials that can be 0 or 1.

In [10]:
x = scipy.stats.binom.rvs(p=0.5, n=1, size=100)
print(x)

[0 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1
 0 1 1 1 0 1 1 0 1 0 0 0 1 1 1 1 0 1 0 0 1 1 0 1 0 0 0 1 0 1 1 1 0 0 0 1 1
 0 0 1 0 0 1 1 0 0 0 1 0 1 0 1 0 1 1 1 1 0 1 1 1 0 0]


For continuous random variables, we can obtain the value of the probability distribution or the cumulative distribution function at any point $x$ as follows:

In [11]:
x = scipy.stats.norm.pdf(x=0.1)
print(x)

x = scipy.stats.norm.cdf(x=0.1)
print(x)

0.3969525474770118
0.539827837277029


For a discrete random variable, we can directly calculate the probability of any specific value based on the probability mass function:

In [12]:
x = scipy.stats.binom.pmf(p=0.5, n=10, k=9)
print(x)

0.00976562500000001
